In [1]:
from huggingface_hub import login
login()

In [2]:
from huggingface_hub import model_info
print(model_info('gpt2'))

ModelInfo(id='openai-community/gpt2', author='openai-community', sha='607a30d783dfa663caf39e06633721c8d4cfcd7e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 2, 19, 10, 57, 45, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=19943214, likes=1674, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'jax', 'tflite', 'rust', 'onnx', 'safetensors', 'gpt2', 'text-generation', 'exbert', 'en', 'doi:10.57967/hf/0039', 'license:mit', 'autotrain_compatible', 'endpoints_compatible', 'has_space', 'text-generation-inference', 'region:us'], pipeline_tag='text-generation', mask_token=None, card_data={'language': 'en', 'license': 'mit', 'library_name': None, 'tags': ['exbert'], 'datasets': None, 'metrics': None, 'eval_results': None, 'model_name': None}, widget_data=None, model_index=None, config={'architectures': ['GPT2LMHeadModel'], 'model_type': 'gpt2', 'task_specific_par

In [3]:
from datasets import load_dataset #예제 데이터 셋
from transformers import AutoTokenizer, DataCollatorWithPadding

In [4]:
#사용할 언어모델
checkpoint = "bert-base-uncased"

In [5]:
#tokenizer 정의
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
#데이터셋 로드
raw_datasets = load_dataset("glue", "mrpc")

In [7]:
#데이터셋 불러오기 및 백터화
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [8]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #동적패딩적용 배치당 가장 긴 문자열 기준

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [9]:
#데이터 전처리 과정

# 1. model에게 필요없는 열 삭제
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# 2. label => labels 모델에 입력할때에는 labels로
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# 3. Pytorch 텐서로 변경
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [10]:
#Dataloader
#Pytorch는 Dataloader를 통해서 데이터 입력
from torch.utils.data import DataLoader

In [11]:
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

In [12]:
# DataLoader가 올바르게 되었는지 검사
for batch in train_dataloader:
    break

In [13]:
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 74]),
 'token_type_ids': torch.Size([8, 74]),
 'attention_mask': torch.Size([8, 74])}

In [14]:
#모델 불러오기
from transformers import AutoModelForSequenceClassification

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) #이진분류

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
#model 작동여부 테스트
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6572, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [17]:
#Optimizer 설정
from transformers import AdamW

In [18]:
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\admin\.conda\envs\nlp\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
#스케줄러 설정
from transformers import get_scheduler

In [20]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [21]:
print(num_training_steps)

1377


In [22]:
#gpu로 학습
import torch

In [23]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device #device(type='cuda')로 출력되면 됨.

device(type='cuda')

In [24]:
#진행과정을 보기 위해서 tqdm 라이브러리 사용
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/1377 [00:00<?, ?it/s]

In [25]:
#모델 훈련
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()} #gpu로 데이터 올려놓음
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [26]:
#평가
import evaluate

In [27]:
metric = evaluate.load("glue", "mrpc")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()} #gpu 사용을 위한 코드 model이 gpu를 사용하기 때문에 데이터도 gpu 로
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8700980392156863, 'f1': 0.9100169779286927}